In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load the dataset to examine its structure
file_path = '/content/drive/MyDrive/New Death Data.csv'
data = pd.read_csv(file_path)

# Display the first few rows and the column names
data.head()

,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,metric_id,metric_name,year,val,upper,lower,Latitude,Longitude
0,1,Deaths,298,American Samoa,1,Male,1,<5 years,386,Nutritional deficiencies,1,Number,2015,0.099075,0.168852,0.051656,-14.297124,-170.713148
1,1,Deaths,298,American Samoa,2,Female,1,<5 years,386,Nutritional deficiencies,1,Number,2015,0.166814,0.292678,0.091553,-14.297124,-170.713148
2,1,Deaths,298,American Samoa,1,Male,1,<5 years,386,Nutritional deficiencies,2,Percent,2015,0.015874,0.025005,0.009543,-14.297124,-170.713148
3,1,Deaths,298,American Samoa,2,Female,1,<5 years,386,Nutritional deficiencies,2,Percent,2015,0.027593,0.044761,0.016449,-14.297124,-170.713148
4,1,Deaths,298,American Samoa,1,Male,1,<5 years,386,Nutritional deficiencies,3,Rate,2015,3.458402,5.894090,1.803130,-14.297124,-170.713148


In [ ]:
data.isnull().values.any()

False

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming 'data' is your DataFrame
funnel_data = data[(data['metric_name'] == 'Number')].groupby(['sex_name', 'cause_name']).agg({'val': 'sum'}).reset_index()
funnel_data.sort_values(by='val', ascending=False, inplace=True)
fig_funnel = px.funnel(funnel_data, x='val', y='cause_name', color='sex_name',
                       labels={"val": "Total Deaths", "cause_name": "Cause of Death"},
                       title="Funnel Chart of Sex, Cause, and Total Deaths")
fig_funnel.show()


In [ ]:
map_data = data[(data['metric_name'] == 'Number')].groupby('location_name').agg({'val': 'sum'}).reset_index()
fig_map_2d = px.choropleth(map_data, locations="location_name", locationmode="country names",
                           color="val", hover_name="location_name",
                           labels={"val": "Total Deaths"},
                           title="2D Map of Total Deaths by Region")
fig_map_2d.show()


In [ ]:
data.head()

,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,metric_id,metric_name,year,val,upper,lower,Latitude,Longitude
0,1,Deaths,298,American Samoa,1,Male,1,<5 years,386,Nutritional deficiencies,1,Number,2015,0.099075,0.168852,0.051656,-14.297124,-170.713148
1,1,Deaths,298,American Samoa,2,Female,1,<5 years,386,Nutritional deficiencies,1,Number,2015,0.166814,0.292678,0.091553,-14.297124,-170.713148
2,1,Deaths,298,American Samoa,1,Male,1,<5 years,386,Nutritional deficiencies,2,Percent,2015,0.015874,0.025005,0.009543,-14.297124,-170.713148
3,1,Deaths,298,American Samoa,2,Female,1,<5 years,386,Nutritional deficiencies,2,Percent,2015,0.027593,0.044761,0.016449,-14.297124,-170.713148
4,1,Deaths,298,American Samoa,1,Male,1,<5 years,386,Nutritional deficiencies,3,Rate,2015,3.458402,5.894090,1.803130,-14.297124,-170.713148


In [ ]:
globe_data.head()

,location_name,val,ISO_alpha
0,American Samoa,2.057890e+03,ASM
1,Antigua and Barbuda,3.133121e+03,ATG
2,Arab Republic of Egypt,3.375021e+06,EGY
3,Argentine Republic,1.852129e+06,ARG
4,Australia,8.271158e+05,AUS


In [ ]:
map_data.head()

,location_name,val,ISO_alpha
0,American Samoa,2.057890e+03,ASM
1,Antigua and Barbuda,3.133121e+03,ATG
2,Arab Republic of Egypt,3.375021e+06,EGY
3,Argentine Republic,1.852129e+06,ARG
4,Australia,8.271158e+05,AUS


In [ ]:
map_data.isnull().values.any()

False

In [ ]:
import plotly.graph_objects as go

# Create the base figure
fig_3d = go.Figure()
globe_data = data

# Create a list of unique causes
unique_causes = globe_data['cause_name'].unique()

# Generate a trace for each cause
for cause in unique_causes:
    cause_data = globe_data[(globe_data['metric_name'] == 'Number') & (globe_data['cause_name'] == cause)].groupby('location_name').agg({'val': 'sum'}).reset_index()
    fig_3d.add_trace(
        go.Choropleth(
            locations=cause_data['location_name'],
            locationmode='country names',
            z=cause_data['val'],
            text=cause_data['location_name'] + ": " + cause_data['val'].astype(str),
            colorscale='Viridis',
            autocolorscale=False,
            reversescale=True,
            marker_line_color='darkgray',
            marker_line_width=0.5,
            colorbar_title='Total Deaths',
            visible=False,
            name=cause
        )
    )

# Create a trace for total deaths (all causes)
total_data = globe_data[globe_data['metric_name'] == 'Number'].groupby('location_name').agg({'val': 'sum'}).reset_index()
fig_3d.add_trace(
    go.Choropleth(
        locations=total_data['location_name'],
        locationmode='country names',
        z=total_data['val'],
        text=total_data['location_name'] + ": " + total_data['val'].astype(str),
        colorscale='Viridis',
        autocolorscale=False,
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title='Total Deaths',
        visible=True,  # Set this trace to be initially visible
        name='Total'
    )
)

# Create the dropdown menu
dropdown_buttons = [
    dict(label='Total', method='update', args=[{'visible': [i == len(unique_causes) for i in range(len(unique_causes) + 1)]}])
]

# Add individual causes to the dropdown
for i, cause in enumerate(unique_causes):
    button = dict(label=cause, method='update', args=[{'visible': [i == j for j in range(len(unique_causes) + 1)]}])
    dropdown_buttons.append(button)

# Update layout with dropdown menu
fig_3d.update_layout(
    title="3D Globe Map of Total Deaths by Cause",
    geo=dict(
        projection_type="orthographic",
        showland=True,
        landcolor="rgb(243, 243, 243)",
        oceancolor="lightblue",
        showocean=True,
        lakecolor="lightblue",
        countrycolor="rgb(204, 204, 204)",
        coastlinecolor="darkgray",
    ),
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.1,
        'xanchor': 'left',
        'y': 1.1,
        'yanchor': 'top'
    }]
)

# Display the figure
fig_3d.show()